<a href="https://colab.research.google.com/github/tatooland/keras/blob/master/word2vec_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q PyDrive
!pip install jieba

In [0]:
import tensorflow as tf
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import os
from zipfile import ZipFile

def downloadFromGDriver(fileId):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)                  
  
  
  
def uploadToDriver(file):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': file})
  uploaded.SetContentFile(file)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))


def uncompressZipFile(fileName):
   ds = ZipFile(fileName)
   ds.extractall()
   os.remove(fileName)
   print("process complete")

In [4]:
fileId = '1GRHNEZIct61cEOzeo1MHa7EBZSi7cFuy'
downloadFromGDriver(fileId)

W0828 10:00:39.583370 140231834793856 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

Extracted zip file 1GRHNEZIct61cEOzeo1MHa7EBZSi7cFuy.zip


In [0]:
%ls

adc.json  __MACOSX/  sample_data/  wunianwenji.txt


In [5]:
import jieba
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import skipgrams

with open("./wunianwenji.txt", 'r+', encoding='utf-8') as f:
  contents = f.read()
  splitList = jieba.cut(contents, cut_all=False)
  splitList = list(splitList)
  text = " ".join(splitList)
  
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([text])
  
  word2id = tokenizer.word_index
  id2word = {v: k for k, v in word2id.items()}
  
  
  wids = [word2id[w] for w in text_to_word_sequence(text)]
  #pairs, labels = skipgrams(wids, len(word2id))
  vocab_size = len(word2id) + 1
  embed_size = 300
  pairs, labels = skipgrams(wids, len(word2id))
  '''
  for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(id2word[pairs[i][0]], pairs[i][0], id2word[pairs[i][1]], pairs[i][1], labels[i]))
  '''
    
  print(type(pairs), type(labels))
  
  
  
  '''
  charList = set(splitList)
  spliteListSize = len(splitList)
  charListSize = len(charList)
  char2Index = dict((c, i) for i, c in enumerate(charList))
  index2Char = dict((i, c) for i, c in enumerate(charList))
  i = 0 
  inputCode = []
  charList1H = np.zeros((len(splitList), len(charList)), dtype=np.bool)
  for idx, token in enumerate(splitList):
    #inputCode.append(char2Index[token]) 
    charList1H[i][char2Index[token]] = 1  
    
    if i < 10:
      print(char2Index[token], i)
    i = i + 1
  print(charList1H[0][9714])
  '''
  i = 0
  for ele, l in zip(pairs, labels):
    i = i + 1
    print(ele, l)
    if i == 10:
      break
  

Building prefix dict from the default dictionary ...
I0828 10:00:44.952992 140231834793856 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
I0828 10:00:44.955661 140231834793856 __init__.py:131] Loading model from cache /tmp/jieba.cache
Loading model cost 0.751 seconds.
I0828 10:00:45.706606 140231834793856 __init__.py:163] Loading model cost 0.751 seconds.
Prefix dict has been built succesfully.
I0828 10:00:45.709352 140231834793856 __init__.py:164] Prefix dict has been built succesfully.


<class 'list'> <class 'list'>
[6, 896] 1
[4400, 4579] 1
[605, 1] 1
[301, 727] 0
[260, 22130] 0
[13, 11] 1
[1, 4393] 0
[30, 24164] 0
[16914, 9600] 1
[93, 1] 1


In [0]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        for i in range(vocab_size):
            in_arr1[0,] = valid_word_idx
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import dot
from tensorflow.keras.layers import Dense, Reshape, Embedding
#from tensorflow.keras.layers.embeddings import Embedding
from tensorflow.keras.models import Sequential
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
import numpy as np


word_model_input = tf.keras.Input((1, ))
word_model_emb = Embedding(vocab_size, embed_size, embeddings_initializer='glorot_uniform', input_length=1)(word_model_input)
word_model = Reshape((embed_size, 1))(word_model_emb)

context_model_input = tf.keras.Input((1, ))
context_model_emb = Embedding(vocab_size, embed_size, embeddings_initializer='glorot_uniform', input_length=1)(context_model_input)
context_model = Reshape((embed_size, 1))(context_model_emb)

dot_product = dot([word_model, context_model], axes=1)
dot_product = Reshape((1,))(dot_product)

z = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(dot_product)
model = tf.keras.Model(inputs=[word_model_input, context_model_input], outputs=z)
model.compile(loss='mean_squared_error', optimizer='rmsprop')#mean_squared_error

print(model.summary())


#SVG(plot_model(model, show_shapes=True, show_layer_names=True))


word_target, word_context = zip(*pairs)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")


arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for epoch in range(1, 10):
  loss = 0
  for i in range(1, 15000):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] =labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
  
    if i % 1000 == 0:
      print('epoch {}, Iteration {}, loss={}'.format(epoch, i, loss))
    if i % 5000 == 0:
      sim_cb.run_sim()

print('loss: {}'.format(loss))
  
'''
  for i, elem in enumerate(pairs):
    pair_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    pair_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    Y = np.array(labels, dtype='int32')
    X = [pair_first_elem, pair_second_elem]
    if i % 10000 == 0:
      print('Processed {} (skip_first, skip_second, relevance) pairs'.format(i))
    loss += model.train_on_batch(X, Y)
  
  print('Epoch:', epoch, 'Loss:', loss)
'''

W0828 10:01:27.570672 140231834793856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       7711800     input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 300)       7711800     input_3[0][0]                    
______________________________________________________________________________________________

NameError: ignored

In [0]:
word_target, word_context = zip(*pairs)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print()